In [60]:
from langchain.document_loaders import GoogleDriveLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders.csv_loader import CSVLoader
import glob

# To store the documents across all data sources:
all_documents = []

# 1. Load the .pdf document:


In [69]:
loader = PyPDFLoader("data/principles_of_marketing_book.pdf")
pages = loader.load_and_split()

print(pages[0])

page_content='Principles of Mark eting' metadata={'source': 'data/principles_of_marketing_book.pdf', 'page': 0}


In [62]:
# Add extra metadata to each page:
for page in pages:
    page.metadata["description"] = "Principles of Marketing Book"

In [63]:
# Checking that the metadata has been added:
for page in pages[0:2]:
    print(page.metadata)

# Saving the marketing book pages:
all_documents.extend(pages)

{'source': 'data/principles_of_marketing_book.pdf', 'page': 0, 'description': 'Principles of Marketing Book'}
{'source': 'data/principles_of_marketing_book.pdf', 'page': 1, 'description': 'Principles of Marketing Book'}


# 2. Search the data folder for all of the .csv files:


In [64]:
csv_files = glob.glob("data/*.csv")
# For each .csv file:
for csv_file in csv_files:
    loader = CSVLoader(file_path=csv_file)
    data = loader.load()
    # Saving the data to the all_documents list:
    all_documents.extend(data)

# 3. Extracting 3 marketing reports from Google Drive.


1. `pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib`
2. Create a Google Cloud project or use an existing project.
3. Enable the Google Drive API
4. Authorize credentials for desktop app - https://developers.google.com/drive/api/quickstart/python#authorize_credentials_for_a_desktop_application


In [65]:
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib --quiet

Note: you may need to restart the kernel to use updated packages.


In [66]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = [
    "https://www.googleapis.com/auth/drive.metadata.readonly",
    "https://www.googleapis.com/auth/drive.readonly",
]


def main():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("drive", "v3", credentials=creds)

        # Call the Drive v3 API
        results = (
            service.files()
            .list(pageSize=10, fields="nextPageToken, files(id, name)")
            .execute()
        )
        items = results.get("files", [])

        if not items:
            print("No files found.")
            return
        print("There are files!")
    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f"An error occurred: {error}")


if __name__ == "__main__":
    main()

There are files!


In [67]:
loader = GoogleDriveLoader(
    folder_id="1cSh-R6VoDImdmUXoApD394k1GDQf6mK5",
    token_path="token.json",  # type: ignore
    # Optional: configure whether to recursively fetch files from subfolders. Defaults to False.
    recursive=False,
)

In [68]:
docs = loader.load()
print(f"Loaded {len(docs)} documents from Google Drive")
all_documents.extend(docs)

Loaded 2 documents from Google Drive
